<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / NOTE 4*

---


# 토픽 모델링(Topic Modeling)

* LDA 모델을 이해하고 Gensim을 사용하여 학습할 수 있습니다.
* LDA 추론 결과를 해석할 수 있습니다.


### Warm up
방대한 텍스트 문서들을 어떻게 하면 주제(토픽)별로 구분할 수 있을가요?
바로 **토픽 모델링**을 사용하면 됩니다!

문서는 토픽들의 집합인데 토픽은 단어의 집합으로 볼 수 있습니다. 즉 각 단어들은 토픽을 대표할 수 있고 토픽들의 집합이 문서가 된다고 볼 수 있습니다.
이 토픽들은 직접적으로 데이터에 나타나지 않기 때문에 숨겨져 있다고 볼 수도 있는데 문서들을 읽다보면 자연스레 알게 되는 핵심 내용들 입니다.

토픽모델링을 수행하는 가장 간단한 방법은 LSA를 사용하는 것 입니다. 하지만 LSA는 단어의 정규분포를 가정하기 때문에 정규분포를 따르지 않는 텍스트의 분석에는 한계가 있습니다.

<img src="https://i.imgur.com/J4tFHFZ.png">

그래서 PLSA(Probabilistic Latent Semantic Analysis)라는 기법이 개발되었는데, LSA에 조건부 확률 개념을 사용해 문서 내 토픽이 등장할 확률을 계산합니다.
즉 pLSA에서는 토픽에 대한 문서 분포나 단어의 분포가 확률로 나오게 되는데 LSA는 그렇지 않습니다. pLSA에서는 SVD를 사용하지 않고 파라미터를 찾기 위해 EM알고리즘을 사용합니다.

LDA(Latent Dirichlet Allocation)는 pLSA와 유사하지만 사전분포 개념을 추가하여 성능을 개선시켰습니다.

오늘은 LDA에 대해 알아보고 토픽 모델링을 구현해 보겠습니다.


#### 다음 영상을 시청하세요
- [LDA Topic Models](https://youtu.be/3mHy4OSyRf0)

# 잠재 디리클레 할당 모델(Latent Dirchilet Allocation Models)


LDA를 최초로 발표한 논문에서 LDA모델의 문서 생성 개념을 설명하는 그림을 보겠습니다.
<img src="https://i.imgur.com/gG3Lwuw.png" />

**LDA 모델이 가정**하는 것은 다음과 같습니다.
- 각 **토픽**은 단어들의 분포입니다.(단어들의 발생 비율)
- 각 **문서**는 여러 토픽들을 혼합해 만들어 집니다.
- 각 **단어**는 이 토픽 중 한 토픽에서 선택(sampling)되어 사용됩니다.


그래서 결국 우리가 학습을 통해 추론하고자 하는 것은

$P(전체 토픽 비율, 문서의 토픽 비율, 단어의 토픽 비율 | D)$ 가 됩니다.


이제 LDA를
1. 문서 생성 가정에 대한 **모델** 부분과
2. 주어진 데이터로 모델을 학습해 파라미터들을 추정하는 **추론** 으로 나누어 살펴보겠습니다.


## LDA 모델부분을 살펴보겠습니다

다음 그림은 문서의 생성 프로세스를 보여주는 그림입니다.
<img src="http://i.imgur.com/CEGcfoM.png" /> 

모델의 노드는 확률변수 입니다. 각 엣지는 영향을 받는 의존관계를 나타냅니다.
사각형 박스(plate)는 반복을 의미합니다. 즉 박스 내부의 N은 N번 반복한다는 의미 입니다.
* 다음은 각 변수들에 대한 설명입니다
    - 하이퍼파라미터 K, α, β
        - K: 전체 토픽 수
        - α: 문서의 토픽 분포를 결정, 값이 클수록 문서에 많은 토픽을 할당함, 디리클레분포(Dir) 파라미터
        - β: 토픽의 단어 분포를 결정, 값이 클수록 토픽에 많은 단어를 할당함, 보통 1 근처로 선택해 랜덤하게 분포되도록 유도함, Dir 분포 파라미터
    - D, 전체 문서 수
    - N, 문서 d 의 전체 단어수
    - $ϕ_{k}$, 토픽 k의 단어 비율을 나타내는 값, 토픽의 단어 분포, Dir(β)로 추정
    - $θ_{d}$, 문서 d의 토픽 비율을 나타내는 값, 문서의 토픽 분포, Dir(α)로 추정
    - $z_{d,n}$, 문서 d, n번째 단어에 토픽을 할당
    - $w_{d,n}$, 문서 d, n번째 단어에 단어를 할당, 유일한 관찰값으로 하이퍼파라미터를 제외한 다른 변수들을 추정하는데 사용

그래서, 우리가 추론단계에서 최대화 해야 하는 목적함수가 다음과 같이 얻어집니다.
\begin{align*}
p(&{ \phi  }_{ 1:K },{ \theta  }_{ 1:D },{ z }_{ 1:D },{ w }_{ 1:D })=\\ &\prod _{ i=1 }^{ K }{ p({ \phi  }_{ i }|\beta ) } \prod _{ d=1 }^{ D }{ p({ \theta  }_{ d }|\alpha ) } \left\{ \prod _{ n=1 }^{ N }{ p({ z }_{ d,n }|{ \theta  }_{ d })p(w_{ d,n }|{ \phi  }_{ 1:K },{ z }_{ d,n }) }  \right\}
\end{align*}

그래서 지금 관찰되는 문서들을 생성시킬 수 있는 확률이 가장 높은 파라미터들을 찾는 것이 LDA 추론(학습) 입니다.

모델의 각 부분에 대해 예시들을 살펴 보겠습니다.

- P(ϕ|β), 토픽의 단어 분포는 디리클레 분포를 통해서 추출되는데 예시는 다음과 같습니다. 한 토픽 내 단어들의 확률을 합하면 1이 됩니다.
<img src="https://i.imgur.com/zwiAx7R.png?1" />

- P(θ|α), 문서 당 토픽의 분포도 디리클레 분포를 통해서 얻게 됩니다. 여기서도 한 문서에서 토픽들의 비율을 합하면 1이 됩니다.
<img src="https://i.imgur.com/hApGv5P.png" />

- P(z|θ), 각 단어를 어떤 토픽에서 뽑아야 하는지 결정하는 단계 입니다. 앞서 본 문서내 토픽의 비율에 따라 토픽이 단어 순서별로 할당됩니다. 
<img src="https://i.imgur.com/HRtYDDA.png" />

- P(w|z,ϕ), 마지막으로 z와 ϕ가 주어 졌을 때 실제 단어를 뽑는 프로세스 입니다.
<img src="https://i.imgur.com/LmTfdkc.png" />

## LDA의 추론(Inference) 부분을 살펴 보겠습니다

지금까지는 LDA의 모델에서 변수들과 문서를 생성하는 과정에 대해 알아 보았습니다. 이번에는 실제 우리가 활용할 수 있는 데이터인 $w_{d,n}$ 을 가지고 변수들을 역으로 추정하는 추론과정에 대해 간략히 알아 보겠습니다.

우리가 코퍼스로부터 추론해야 하는 부분은 다음 3가지가 됩니다.
- $z_{d,n}$, 단어별 토픽 할당
- $θ_{d}$, 문서별 토픽 비율
- $ϕ_{k}$, 전체 코퍼스에서 토픽의 단어발생 분포

결국 사후확률(posterior)
\begin{equation}
p(\theta, \phi, z|w, \alpha, \beta) = {p(\theta, \phi, z, w|\alpha, \beta) \over p(w|\alpha, \beta)}
\end{equation}

 를 최대로 만드는 θ,ϕ,z를 찾아야 합니다.

여기서 분모부분 $p(w)$ 은 모든 가능한 θ,ϕ,z 를 모두 고려한 단어w(marginal p(w))의 확률을 말하는데, θ,ϕ,z를 모두 관찰하는게 불가능하기 때문에 근사적으로 사후확률을 추론하는 알고리즘인 깁스 샘플링(Gibbs sampling) 같은 방법을 사용하게 됩니다.

#### 하이퍼파라미터 α, β
α, β 는 추론의 대상이 아닙니다 두 디리클레분포를 모양을 결정하는 하이퍼파라미터 입니다. 
그중 α,B 값에 따라어 디리클레 분포의 모양이 변하는 모습을 보면 다음과 같습니다. y축이 토픽의 비율, x축이 토픽들, 각 박스칸의 숫자가 문서라고 할 때, α 값이 커질 수록 문서에 토픽들이 균등하게 분포되는 것을 확인할 수 있습니다.

즉 문서별로 적은 토픽을 할당하려 한다면 α 값을 낮추어야 하겠습니다. 관례적으로 α 값은 0보다 작게 잡습니다.
반면에 β를 통해 $ϕ_{k}$를 추정할 경우 보통 1이하로 잡지는 않습니다.

<img src="https://i.imgur.com/LJ6aqSJ.png" width="600" />

<img src="https://miro.medium.com/max/1400/1*_NdnljMqi8L2_lAYwH3JDQ.gif" width="600" />


#### 깁스샘플링(Gibbs Sampling)
이제 앞서 설명한 사후확률인 $p(\theta, \phi, z|w, \alpha, \beta)$ 를 계산해야 하는데 여러 
근사 알고리즘 중 Collapsed gibbs sampling 으로 구하는 방법을 알아보겠습니다. 깁스 샘플링 시 z를 구하고 θ,ϕ는 구하지 않는데, 그 이유는 찾아낸 z 값으로 나중에 문서내 토픽 분포를 찾아 θ를 구하고, 토픽내 단어 분포를 사용해 ϕ 를 구할 수 있게 되기 때문 입니다.

깁스샘플링을 수식으로 나타내면 다음과 같습니다.

$p({ z }_{ i }=j|{ z }_{ -i },w)$

이 식은 내가 할당하고자 하는 i번째 단어에 토픽 j가 할당될 확률을 나타내는데, $z_{i}$를 빼고는 모두 토픽이 할당되어 있는 상태를 의미합니다. $z_{-i}$ 는 i번째 단어를 제외하고 할당되어 있는 모든 단어의 토픽정보를 의미하고, w는 주어진 단어들로 우리가 알고 있습니다.

다음 그림은 깁스 샘플링 과정을 보여줍니다. 우선 모든 단어에는 토픽(노랑, 빨강, 파랑)이 랜덤하게 할당되어 있고 모든 문서에서 한 단어씩 선택하며 진행하는데, 다음 과정을 반복합니다.

1. 선택된 단어에 할당된 토픽은 지우고, 다른 모든 단어들의 토픽 할당 상황을 고려하여, **선택된 단어에 토픽을 할당**합니다. 가능한 확률과 랜덤성이 동시에 작용합니다.
2. 다음 단어로 이동하여 1을 반복합니다.

<img src="https://i.imgur.com/PrB0GUW.png" />

위 반복과정에서 선택된 단어에 토픽을 할당하는 것은 다음 식으로 나타낼 수 있습니다.

$p({ z }_{d, i }=j|{ z }_{ -i },w)=\frac { { n }_{ d,k }+{ \alpha  }_{ j } }{ \sum _{ i=1 }^{ K }{ ({ n }_{ d,i }+{ \alpha  }_{ i }) }  } \times \frac { { v }_{ k,{ w }_{ d,n } }+{ \beta  }_{ { w }_{ d,n } } }{ \sum _{ j=1 }^{ V }{ ({ v }_{ k,j }+{ \beta  }_{ j }) }  }=AB$

- $n_{d,k}$, 문서d 내 토픽k 갯수
- $v_{k,{w}_{d,n}}$, 토픽k에 존재하는 단어$w_{d,n}$의 수
- ${\alpha}_{k}$, ${\beta}_{w_{d,n}}$, 디리클레 파라미터로 
- **A, 문서d와 토픽k의 연관성**
- **B, 문서d의 단어n이 토픽k와의 연관성**

이 과정을 코퍼스 전체 단어에 대해 한 번씩 진행한 것을 반복수 라고 할 때 이 반복수를 늘릴 수록 단어들의 토픽 할당이 수렴하기를 기대하며 추론을 진행합니다.


### LDA가 어떻게 작동하는지 매우 간단히 살펴볼 수 있는 사이트가 있습니다.


- [LDA Topic Modeling](https://lettier.com/projects/lda-topic-modeling/)

- [Your Guide to Latent Dirichlet Allocation](https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d)

<img src="https://miro.medium.com/max/2000/1*f7ODdUPZtkcWUNcT0CJDzw.png">



# Gensim을 사용하여 LDA 모델을 추정해 봅시다

### 러시아 문학의 거장 톨스토이와 도스토예프스키 작품 비교

러시아 문학의 거장 톨스토이와 도스토예프스키의 작품을 읽어 보셨나요?
전쟁과평화, 안나카레니나 vs 죄와벌, 카라마조프의 형제들, 백치, 악령

두 문호들은 같은 시대(1800년대)를 살아왔지만 단 한 번도 만난적이 없다고 합니다.
그리고 지향하는 관점과 색깔, 작품세계가 완전히 다르다고 하는데..
그들의 소설을 LDA을 사용하여 토픽모델링을 수행해 보고 작품을 비교해 보도록 합시다.

In [ ]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [ ]:
gensim.__version__

### 문학 데이터

데이터로 쓰일 소설은 [Project Gutenberg](www.gutenberg.org) 에서 가져와 작은 텍스트 파일로 분해하여 제공됩니다.

다운로드 받은 파일의 경로를 잘 지정해 주세요.
[다운로드 링크](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/nlp_novels/nlp_novels.zip)

In [ ]:
path = './data'

### 텍스트 파일 전처리

`tokenize` 함수는 자유롭게 수정 보완 하여 사용하세요

In [ ]:
# 파이썬에서는 토큰을 ''.split 를 사용하여 나누었고
# Spacy에서는 lemma_ 를 사용하였습니다.
# 이번에 사용할 Gensim에서는 simple_preprocess를 사용해 보겠습니다.

STOPWORDS = set(STOPWORDS).union(set(['CHAPTER', 'said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text, max_len=25) if token not in STOPWORDS]

In [ ]:
str1 = 'Bananachachabang and  st1 prince   coco live there! !@#$ % Pneumonoultramicroscopicsilicovolcanoconiosis'
tokenize(str1)

In [ ]:
# preprocess_string을 사용할 수도 있습니다.
# https://radimrehurek.com/gensim/parsing/preprocessing.html
from gensim.parsing.preprocessing import preprocess_string, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_non_alphanum, strip_short

gensim_filter = [lambda x: x.lower()
                , strip_multiple_whitespaces # 두개 이상 공백은 제거합니다
                , strip_numeric
                , remove_stopwords
                , strip_non_alphanum # 영문자, 숫자가 아닌경우 제거합니다
                , strip_short # 단어길이가 3 이하인 경우 제거합니다
                ]

# gensim filter를 사용합니다
preprocess_string(str1, gensim_filter)

In [ ]:
import os

# 문서를 불러올 때 파이썬 제네레이터를 사용합니다.
# 제네레이터는 메모리를 효율적으로 사용하기 위한 도구 입니다.
# 함수에서 return 대신 yield를 사용하면 함수는 제네레이터가 되고 yield에는 값을 지정합니다.
# 제네레이터는 next() 함수로 호출이 되는데 호출될 때 마다 yield 까지 코드를 실행하며
# yield에서 값을 발생(generate) 시킵니다.

# 그래서 이 함수는 각 문서 파일들의 토큰을 한 번씩 발생시키는 제네레이터가 됩니다.

def stream_data(path_to_data): 
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
#                     tokens = preprocess_string(text)
                    tokens = tokenize(text) # 토큰화
                yield tokens # 제네레이터

In [ ]:
# 토큰들이 들어있는 generator 
streams = stream_data(path)
streams

In [ ]:
tokens = []
for doc in streams:
    tokens.append(doc)


In [ ]:
len(tokens)

In [ ]:
tokens[0][0:5]

In [ ]:
len(tokens[0])

In [ ]:
# 확장자를 제외한 파일명을 불러옵니다.
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [ ]:
titles[:5]

In [ ]:
len(titles), len(tokens)

### 분석에 사용할 데이터프레임을 만들겠습니다

In [ ]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [ ]:
df.head()

In [ ]:
df = df.reset_index()

In [ ]:
df['author'] = df['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [ ]:
df['author'] = df['author'].map({'Tolstoy':0, 'Dostoevsky':1})

In [ ]:
df.author.value_counts()

In [ ]:
df.head()

### Gensim으로 LDA를 진행해 보겠습니다

In [ ]:
# 코퍼스에 있는 모든 단어를 gensim 딕셔너리로 만듭니다
gensim_dic = corpora.Dictionary(stream_data(path))

In [ ]:
# 전체 문서 갯수 입니다
gensim_dic.num_docs

In [ ]:
# token2id 메소드는 토큰(단어)의 할당된 인덱스를 리턴합니다
gensim_dic.token2id['book']

In [ ]:
# doc2bow는 텍스트를 BoW표현으로 나타냅니다
gensim_dic.doc2bow(tokenize("I read a great many Russian book book book"))

In [ ]:
type(gensim_dic)

In [ ]:
# 전체 단어의 종류입니다
len(gensim_dic.keys())

In [ ]:
# gensim에서는 극단적인 토큰을 필터링 할 수 있는 방법을 제공합니다
gensim_dic.filter_extremes(no_below=6, no_above=0.50)

In [ ]:
len(gensim_dic.keys())

In [ ]:
# 전체 코퍼스의 BoW 표현을 만들어 봅시다
corpus = [gensim_dic.doc2bow(text) for text in stream_data(path)]

In [ ]:
corpus[0][:10]

In [ ]:
# 전체 문서(파일) 수 입니다
len(corpus)

In [ ]:
# 문서12번(전체 의 BoW를 보겠습니다
corpus[12][:10]

In [ ]:
%%time
# LDA 모델을 학습해 봅시다.
lda = LdaMulticore(corpus = corpus
                   , id2word = gensim_dic # id-단어 맵핑
                   , random_state = 2
                   , num_topics = 6
                   , passes = 100 # 전체 코퍼스 반복 빈도 제어
                   , workers = 10 # 사용 CPU 코어수
                   , alpha = 1 # 디리클레 분포에 대한 하이퍼파라미터
                   , eta = 'auto' # 디리클레 분포에 대한 하이퍼파라미터 (beta)
                  )

In [ ]:
lda.print_topics()

In [ ]:
lda.print_topics()[0][1]

In [ ]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [ ]:
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
for id, t in enumerate(topics):
    print(f"Topic {id}: ", t, end="\n\n")

# LDA 결과를 해석해 봅시다

### pyLDAvis를 사용한 토픽 시각화

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

### 토픽모델링 결과 해석을 위해 도스토예프스키와 톨스토이의 작품들과 등장인물에 대해 간략히 소개하겠습니다

### 도스토예프스키의 작품들
#### 죄와벌: 돈에 굶주린 전당포 노인을 도끼로 살해한 가난한 청년 대학생과 창녀 소냐를 통해 인간의 죄와 그에 따른 죄의 댓가, 원초적 사랑, 고통과 자기 희생 등을 감명깊게 그린 걸작입니다.
등장인물이 매우 많고 복잡한, 도스토예프스키의 대표적인 소설입니다.
등장인물로 주인공 **raskolnikov**,  연인 sonia, 동생 dounia, 친구 razumihin(두냐와 결혼), 두냐의 예전 약혼남 루신(Pyotr Petrovich Luzhin) 등이 있습니다.

#### 까라마조프가 형제들: 도스토예프스키 만년의 대표작입니다. 부도덕하고 냉소적인 아버지 카라마조프에게는 3명의 아들이 있습니다.
카라마조프가 아버지 표도르(Fyodor Pavlovich Karamazov), 아버지에게 극도의 증오심을 품고 있는 장남 드미트리(Dmitri Fyodorovich Karamazov),
지식탐구에 몰두하는 차남 이반(Ivan), 신앙심이 깊은 막내 알료샤(**Alyosha**) 이들을 둘러싼 사건들을 통해 종교, 사상, 삶의 본질 등을 탐구하고 있습니다.
다른 주요 등장인물로 표도르 사생아이며 하인인 스메르쟈코프(smerdyakov)
드미트리 약혼녀이자 이반의 연인인 카테리나(katerina), 표도르와 드미트리의 연인관계였던 그루센카(grushenka)가 있습니다.

#### 백치(The Idiot): 인간의 이면과 이중성을 잘 나타낸 작품입니다.
주인공인 젊은 귀족 미쉬킨(**muishkin**)은 순진하고 어리숙한 성격으로 사람들이 백치로 규정하지만
사실은 인간 이면을 꿰둟어보는 능력을 가지고 있었습니다. 기행을 일삼는 나스타샤(**nastasia**)를 사랑하지만 친구이자
연적인 로고진(**rogojin**)은 나중에 나스타샤를 살해합니다.
한편 미쉬킨은 혼담이 오가는 아글라야(**aglaya**)와 나스타샤 사이에서 누구를 선택할지 결론을 내지 못하는 백치같은 모습을 보여주기도 합니다.
그로인해 결국 미쉬킨, 아글라야, 나스타샤 모두 불행을 맞이하게 됩니다.
그외 친척 리자베타(lizabetha), 예판친(epanchin)등이 추가적인 등장인물 입니다

#### 악령(The Possessed): 수수께끼에 싸인 젊은 귀족 니꼴라이 스따브로긴과 그를 둘러싼 비밀 혁명 조직의 일당들이 초래하는
비극적인 사건들을 통해, 서구와 러시아, 자유주의와 허무주의, 무신론과 인신사상, 슬라브주의와 러시아 정교,
세대 간의 갈등, 구원과 속죄의 문제 등 당대 러시아의 주요 화두들과 도스토예프스키가 평생에 걸쳐 천착했던 주제들을 심도 있게 다루고 있습니다.
주인공 스타브로긴(**stavrogin**), 사교계의 큰손 어머니 바르바라(varvara **petrovna**),
어머니의 친구이며 가정교사 스테판(stepan), 스테판 아들로 문제적인 인물 표트르(Pyotr Stepanovitch)
바르바라 수양딸 다샤와 그의 남매 샤토프(shatov)가 등장인물 입니다.

### 톨스토이의 작품들
#### 전쟁과 평화: 톨스토이의 3대 걸작 중 하나로 전쟁과 평화의 반복을 보여줍니다.
이 소설의 배경으로 아우스터리츠 전투와, 1812년 나폴레옹(**napoleon**) 의 러시아 원정이 있습니다. 이 작품의 등장인물은 559명으로
상당히 많습니다. 그중 가장 대표적인 등장인물 6인은 다음과 같습니다.
볼콘스키 가문의 안드레이(Andrew)와 여동생 마리야(Mary), 베주호프 가문의 사생아 피예르(pierre), 그의 아내인 쿠라긴 가문의 옐렌
로스토프(rostóv) 가문의 니콜라이와 그의 여동생 나타샤(natásha)도 있습니다.

#### 안나 카레니나: 도스토예프스키도 톨스토이의 대표작 안나 카레니나에 대해 찬사를 아끼지 않았다.
이 작품은 불륜에 빠진 주인공 안나(**Anna**)와 젊은 장교 브론스키(**Vronsky**)와의 비극과
또다른 주인공 레빈(**levin**)과 키티(**Kitty**) 부부의 순수한 사랑이 대비되는 소설입니다.
등장인물로 안나의 남편 알렉세이(Alexey **Alexandrovitch**), 안나 오빠인 스테판(Stepan)과 아내 돌리(Dolly)가 있습니다.


### 만들어진 모델과 문서들에 대해 알아보겠습니다

In [ ]:
# 문서3이 연관된 토픽과 그에 해당하는 단어의 비율 입니다
lda[corpus[3]]

In [ ]:
# 문서들의 토픽 분포
dist = [lda[doc] for doc in corpus]

In [ ]:
dist[3]

In [ ]:
dist = [lda[doc] for doc in corpus]

# 문서에 포함되지 않은 토픽은 0으로 업데이트합니다
def update(doc):
        d_dist = {k:0 for k in range(0,6)} # k: topics
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_dist = [update(doc) for doc in dist]

In [ ]:
new_dist[3]

In [ ]:
df.head()

In [ ]:
topics

In [ ]:
new_dist[:5]

In [ ]:
df = pd.DataFrame.from_records(new_dist, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [ ]:
df.head(20)

In [ ]:
df.groupby('author').mean()

# 토픽 수는 어떻게 결정할까요?

토픽 수에 대한 평가를 위해 Perplexity, Topic Coherence 를 측정하는 방법이 있습니다.

그 중 Umass coherence 방법은 같은 토픽 내에서 상위 단어간에 나타나는 문서 수를 측정해 coherence를 측정합니다.

$\text{score}_{\text{UMass}}(w_i, w_j) =
    \log
    \frac{D(w_i, w_j) + 1}{D(w_i)}$

D(w)는 단어 w가 나타나는 문서의 수 이며, D($w_{i}, w_{j}$) 는 $w_{i}$, $w_{j}$가 동시에 나타나는 문서의 수 입니다.

coherence score 값이 0에 가까운 토픽 수를 선택하는 것이 좋다고 합니다.

하지만 현실적으로 토픽수에 정답은 없습니다. 만들어진 토픽들을 보고 직접 판단을 하는 것이 중요합니다.


In [ ]:

from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    여러 토픽수에 대해 u_mass coherence를 계산합니다.

    Parameters: 
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : 최대 토픽수
    passes: 전체 코퍼스 학습 반복 빈도를 제어합니다

    Returns:
    -------
    coherence_values : 각 토픽 수에 대한 u_mass coherence 값 입니다.
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        print(f'Pass #{iter_} Evaluating model with : ', end = " ")
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=10)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass', processes=-1)
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })
            
            print(f'{num_topics}', end = " ")
        print('topics')
    return coherence_values

In [ ]:
%%time
# 시간이 오래걸리니 주의하세요
coherence_values = compute_coherence_values(dictionary=gensim_dic, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=25, 
                                                        step=2,
                                                        passes=25)

In [ ]:
coherence_values[0]

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.sort_values(by=['coherence_score'])

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

## 참고자료

- [Topic Modeling,LDA](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/06/01/LDA/): LDA에 대해 잘 정리된 글
- [Topic Modelling Using Latent Dirichlet Allocation — Algorithm Description and Use Case](https://medium.com/@smart.sam97/topic-modelling-using-latent-dirichlet-allocation-algorithm-description-and-use-case-c46004c562ef)
- [Gensim](https://radimrehurek.com/gensim/): 토픽모델링에 관한 파이썬 패키지
- [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): Gensim을 사용한 LDA 예시
- [PyLDAvis](https://github.com/bmabey/pyLDAvis): 토픽모델링 시각화 툴
- [Latent Dirichlet Allocation](https://jmlr.org/papers/volume3/blei03a/blei03a.pdf)

#### Topic Coherence
- [Topic Coherence To Evaluate Topic Models](http://qpleple.com/topic-coherence-to-evaluate-topic-models/)
- [How does topic coherence score in LDA intuitively makes sense?](https://stats.stackexchange.com/questions/375062/how-does-topic-coherence-score-in-lda-intuitively-makes-sense)
- [Optimizing Semantic Coherence in Topic Models](http://dirichlet.net/pdf/mimno11optimizing.pdf)

#### 참고 영상
- [Topic Modeling Part 2 (Kor)](https://youtu.be/WR2On5QAqJQ)
- [Topic Modeling with Latent Dirichlet Allocation in Python](https://www.youtube.com/watch?v=NYkbqzTlW3w)


